In [3]:
import math
import numpy
from matplotlib import pyplot
%matplotlib inline

In [4]:
# Set the font family and size to use for Matplotlib figures.
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [5]:
# Set parameters.
g = 9.81 # gravitational acceleration (m.s^{-2})
vt = 30.0 # trim velocity (m.s)
CD = 1.0/40 # drag coefficient
CL = 1.0 # lift coefficient

# Set initial conditions.
v0 = vt # start at the trim velocity
theta0 = 0.0 # trajectory angle
x0 = 0.0 # horizontal position
y0 = 1000.0 # vertical position (altitude)

In [ ]:
def rhs_phugoid(u, CL, CD, g, vt):
    """
    Returns the right-hand side of the phugoid system of equations.
    
    Parameters
    -----------
    u : list or numpy.ndarray
        Solution at the previous time step 
        as a list or 1D array of four floats.
    CL : float
        Lift coefficient.
    CD : float
        Drag coefficient.
    g : float
        Gravitational acceleration.
    vt : float
        Trim velocity
        
    Returns
    --------
    rhs : numpy.ndarray
        The right-hand side of the system
        as a 1d array of four floats.
    """
    v, theta, x, y = u
    rhs = numpy.array([-g*math.sin(theta)-CD/CL * g//vt**2*v**2,
                      -g*math.cos(theta)/v + g/vt**2*v,
                       v*math.cos(theta), 
                       V*math.sin(theta)])
        